# Routing by semantic similarity

With LCEL you can easily add custom routing logic to your chain to dynamically determine the chain logic based on user input. All you need to do is define a function that given an input returns a Runnable.

One especially useful technique is to use embeddings to route a query to the most relevant prompt. Here’s a very simple example.

In [2]:
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
model_config_path = os.path.abspath(os.path.join('../custom_llms/'))
sys.path.insert(0, module_path)
sys.path.insert(0, model_config_path)

from custom_llms.qwen_llm import DashLLM
model = DashLLM()

from langchain.embeddings import DashScopeEmbeddings
embeddings = DashScopeEmbeddings(
    model="text-embedding-v1", dashscope_api_key=os.environ["DASHSCOPE_API_KEY"]
)

In [3]:
from langchain.prompts import PromptTemplate
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

math_template = """You are a very good mathematician. You are great at answering math questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

prompt_templates = [physics_template, math_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)


def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    print("Using MATH" if most_similar == math_template else "Using PHYSICS")
    return PromptTemplate.from_template(most_similar)


chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | model
    | StrOutputParser()
)

In [4]:
print(chain.invoke("What's a black hole"))

Using MATH
A black hole is an object with a gravitational pull so strong that nothing can escape from it, not even light. This means that anything that gets too close to a black hole will be pulled in and never seen again.
Black holes are formed when massive stars run out of fuel and collapse in on themselves. The intense gravity of the collapsed star causes all of its matter to be compressed into an incredibly small space, creating a singularity - a point of infinite density where the laws of physics as we know them break down.
The boundary around a black hole beyond which nothing can escape is called the event horizon. Once something crosses the event horizon, it is effectively lost forever, as it cannot send any information back outside the black hole.


In [5]:
print(chain.invoke("What's a path integral"))

Using MATH
A path integral is a mathematical concept used in quantum mechanics to calculate the probability amplitude for a system to go from one state to another. It involves summing over all possible paths that the system could take between the two states, weighted by a phase factor that depends on the action of the system along each path. The path integral can be thought of as a way of averaging over all possible paths that the system could take, with the weight given to each path being determined by its action.
